# Convergence analysis for a cantilever beam


This notebook presents convergence analysis using the Finite Element
Method for a cantilever beam.

The following figure shows a cantelever beam under a tip load $P$. The beam is of length $l$;
moment of inertia $I$; height $2c$ and material properties corresponding
to Poissons ratio and Young modulos $\nu$ and $E$ respectively.

<center><img src="../img/beam.jpg" alt="Cantilever beam" style="width:600px"></center>

The analytic solution is given by (see Timoshenko):

$$u =  - \frac{P}{{2EI}}{x^2}y - \frac{{\nu P}}{{6EI}}{y^3} + \frac{P}{{2IG}}{y^3} + \left( {\frac{{P{l^2}}}{{2EI}} - \frac{{P{c^2}}}{{2IG}}} \right)y$$

$$v = \frac{{\nu P}}{{2EI}}x{y^2} + \frac{P}{{6EI}}{x^3} - \frac{{P{l^2}}}{{2EI}}x + \frac{{P{l^3}}}{{3EI}}$$

$${\varepsilon _{xx}} = \frac{{\partial u}}{{\partial x}} \equiv  - \frac{P}{{EI}}xy$$

$${\varepsilon _{yy}} = \frac{{\partial v}}{{\partial y}} \equiv \frac{{\nu P}}{{EI}}xy$$

$${\gamma _{xy}} = \frac{{\partial u}}{{\partial y}} + \frac{{\partial v}}{{\partial x}} \equiv \frac{P}{{2IG}}\left( {{y^2} - {c^2}} \right)$$

The following snippet compute the solution for an array of points $x$, $y$.

In [ ]:
from __future__ import division, print_function
import numpy as np
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
from matplotlib import rcParams
from solidspy.preprocesor import rect_grid
import solidspy.postprocesor as pos
import solidspy.assemutil as ass
import solidspy.solutil as sol

ModuleNotFoundError: No module named 'numpy'

In [2]:
%matplotlib notebook


rcParams['figure.figsize'] = 6, 4
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 12

In [3]:
def beam_sln(x, y, nu, P, E, I, L, h):
    """Compute the solution for a cantilever beam

    Parameters
    ----------
    x : ndarray (float)
        Array with x coordinates.
    y : ndarray (float)
        Array with y coordinates.
    nu : float, (-1, 0.5)
        Poisson coefficient.
    P : float
        Applied force at the end of the beam.
    E : float, >0
        Young modulus.
    I : float, >0
        Moment of inertia.
    L : float, >0
        Length of the beam.
    h : float, >0
        Height of the beam.

    Returns
    -------
    u : ndarray (float)
        Horizontal displacement at the nodes.
    v : ndarray (float)
        Vertical displacement at the nodes.
    exx : ndarray (float)
        xx component of the strain tensor.
    eyy : ndarray (float)
        yy component of the strain tensor.
    gammaxy : ndarray (float)
        xy component of the strain tensor.
        
    References
    ----------
    .. [1] Timoshenko, S. & Goodier, J., 1970. Theory of Elasticity,
        McGraw-Hill, 3rd Ed.

    """
    G = E/(2*(1 + nu))
    c = h/2
    C1 = -P/(2*E*I)
    C2 = -(nu*P)/(6*E*I)
    C3 = P/(2*I*G)
    C4 = (P*L**2)/(2*E*I)
    C5 = -(P*c**2)/(2*I*G)
    C6 = C4 + C5
    C7 = (nu*P)/(2*E*I)
    C8 = P/(6*E*I)
    C9 = -(P*L**2)/(2*E*I)
    C10 = (P*L**3)/(3*E*I)
    B1 = -P/(E*I)
    B2 = (nu*P)/(E*I)
    B3 = P/(2*I*G)
    u = C1*y*x**2 + C2*y**3 + C3*y**3 + (C5 + C6)*y
    v = C7*x*y**2 + C8*x**3 + C9*x + C10
    exx = B1*x*y
    eyy = B2*x*y
    gammaxy = B3*(y**2 - c**2)

    return u, v, exx, eyy, gammaxy

In [4]:
nu, P, E, I, L, h = 0.3, -50, 1000, 42.67, 24, 4
x = np.linspace(0, L, 100)
y = np.linspace(-h, h, 100)
x, y  = np.meshgrid(x, y)
u, v, exx, eyy, gammaxy  = beam_sln(x, y, nu, P, E, I, L, h)

The particular solution for parameters $E=1000.0$, $P=-50$ $\nu=0.30$, $I=42.67$,
$l=24$ and $2c=8$ is shown below:

In [5]:
plt.figure()
plt.contourf(x, y, u, 10)
plt.axis("image")


<IPython.core.display.Javascript object>

<div id='ebff90f2-4ea7-4801-8039-b8717fc2a3cc'></div>

(0.0, 24.0, -4.0, 4.0)

In [6]:
plt.figure()
plt.contourf(x, y, v, 10)
plt.axis("image")

<IPython.core.display.Javascript object>

<div id='f424ae03-99b3-487b-a84f-774f251589d3'></div>

(0.0, 24.0, -4.0, 4.0)

We also want to analyze this problem using the FEM with bilineal square elements under plane stress conditions.

Since this is a structured mesh it is easily generated in a simple function.

In [7]:
P = -50
E = 1000
nu = 0.3
L = 24
h = 8
I = 42.67
analytic = True
niter = 8
err = np.zeros((niter))
mats = np.array([[E, nu], [E, nu]])
fname = "error_vs_h.txt"
fid = open(fname, "w")
fid.write("Iteration, Elements, h, error\n")
fid.close()

In [8]:
# y, x = np.mgrid[-4:4:(3)*1j, -12:12:(13)*1j]


Notice that in the finite elment models the load is applied as nodal point loads of the same magnitud distributed along all the nodes at $x=-12$. This implies a uniform load distribution instead of the parabolic load that is consistent with the shear stress in the analytic solution.

In [12]:
for cont in range(1, niter + 1):

    print("Starting iteration %i, " % cont + "h=%g, %i elements" % (8/2**(cont - 1), 3*4**(cont - 1)))
    nx = 3*2**(cont - 1)
    ny = 2**(cont - 1)
    x, y, els = rect_grid(L, h, nx, ny)
    nodes = np.zeros(((nx + 1)*(ny + 1), 5))
    nodes[:, 0] = range((nx + 1)*(ny + 1))
    nodes[:, 1] = x
    nodes[:, 2] = y
    nodes[x==L/2, 3] = -1
    nodes[nx*(ny//2 + 1) - 1, 4] = -1    
    loads = np.zeros((ny + 1, 3))
    loads[:, 0] = nodes[x==-L/2, 0]
    loads[:, 2] = P/ny
    
    # Assembly
    DME, IBC, neq = ass.DME(nodes, els)
    print(DME)
    
    stiff = ass.assembler(els, mats, nodes, neq, DME)
    rhs = ass.loadasem(loads, IBC, neq)
    
    # Solution
    disp = sol.static_sol(stiff, rhs)
    disp_comp = pos.complete_disp(IBC, nodes, disp)
    
    # Interpolation and error evaluation
    if cont > 1:
        disp_interp = np.column_stack([u_interp, v_interp])
        aux = np.linalg.norm(disp_interp - disp_comp)
        err[cont - 1] = aux/np.linalg.norm(disp_comp)
        fid = open(fname, "a")
        fid.write("%i, %i, %g, %g\n" %
                 (cont, 3*4**(cont - 1), 8*1/2**(cont - 1), err[cont-1]))
        fid.close()
    
    x_new, y_new, _ = rect_grid(L, h, 2*nx, 2*ny)
    u_interp = griddata((x, y), disp_comp[:, 0], (x_new, y_new))
    v_interp = griddata((x, y), disp_comp[:, 1], (x_new, y_new))

Starting iteration 1, h=8, 3 elements
[[ 0  1  2  3  8  9  6  7  0  0  0  0  0  0  0  0  0  0]
 [ 2  3  4 -1 10 11  8  9  0  0  0  0  0  0  0  0  0  0]
 [ 4 -1 -1  5 -1 12 10 11  0  0  0  0  0  0  0  0  0  0]]
Starting iteration 2, h=4, 12 elements
[[ 0  1  2  3 15 16 13 14  0  0  0  0  0  0  0  0  0  0]
 [ 2  3  4  5 17 18 15 16  0  0  0  0  0  0  0  0  0  0]
 [ 4  5  6  7 19 20 17 18  0  0  0  0  0  0  0  0  0  0]
 [ 6  7  8  9 21 -1 19 20  0  0  0  0  0  0  0  0  0  0]
 [ 8  9 10 11 22 23 21 -1  0  0  0  0  0  0  0  0  0  0]
 [10 11 -1 12 -1 24 22 23  0  0  0  0  0  0  0  0  0  0]
 [13 14 15 16 27 28 25 26  0  0  0  0  0  0  0  0  0  0]
 [15 16 17 18 29 30 27 28  0  0  0  0  0  0  0  0  0  0]
 [17 18 19 20 31 32 29 30  0  0  0  0  0  0  0  0  0  0]
 [19 20 21 -1 33 34 31 32  0  0  0  0  0  0  0  0  0  0]
 [21 -1 22 23 35 36 33 34  0  0  0  0  0  0  0  0  0  0]
 [22 23 -1 24 -1 37 35 36  0  0  0  0  0  0  0  0  0  0]]
Starting iteration 3, h=2, 48 elements
[[  0   1   2   3  27  28  

In [23]:
DME

array([[    0,     1,     2, ...,     0,     0,     0],
       [    2,     3,     4, ...,     0,     0,     0],
       [    4,     5,     6, ...,     0,     0,     0],
       ...,
       [98424, 98425, 98426, ...,     0,     0,     0],
       [98426, 98427, 98428, ...,     0,     0,     0],
       [98428, 98429,    -1, ...,     0,     0,     0]])

In [13]:
data = np.loadtxt("error_vs_h.txt", skiprows=1, delimiter=",")
els = data[:, 1]
h = data[:, 2]
err = data[:, 3]
fig = plt.figure()
ax1 = fig.add_subplot(111)
ax1.loglog(h, err, '-bo')
plt.xlabel(r"Element size: $h$")
plt.ylabel(r"Relative error: $\frac{\Vert u - u_h \Vert}{\Vert u \Vert}$",
           fontsize=14)
xticks, xlabels = plt.xticks()
plt.grid(which="both", alpha=0.3, lw=0.5)
ax2 = ax1.twiny()
ax2.set_xscale("log")
ax2.set_xlim(ax1.get_xlim())
ax2.set_xticks(xticks[1:-1])
labels = 192/4**np.log2(xticks[1:-1])
ax2.set_xticklabels(["%g"%label for label in labels])
ax2.xaxis.set_ticks_position("none")
plt.xlabel("Number of elements")
plt.tight_layout()

<IPython.core.display.Javascript object>

<div id='6a684049-0b25-49ec-ae7d-0348fed6b5da'></div>

## Refences

1. Timoshenko, S. P. J. N. "Goodier, Theory of elasticity." McGraw-Hill Book Co., Inc., New York (1970).

The following cell gives format to the notebook.

In [16]:
from IPython.core.display import HTML
def css_styling():
    styles = open('/Users/yashgandhe/Coding/SolidsPy/examples/custom_barba.css', 'r').read()
    return HTML(styles)
css_styling()

<link href='http://fonts.googleapis.com/css?family=Fenix' rel='stylesheet' type='text/css'>
<link href='http://fonts.googleapis.com/css?family=Alegreya+Sans:100,300,400,500,700,800,900,100italic,300italic,400italic,500italic,700italic,800italic,900italic' rel='stylesheet' type='text/css'>
<link href='http://fonts.googleapis.com/css?family=Source+Code+Pro:300,400' rel='stylesheet' type='text/css'>
<style>
/* Based on Lorena Barba template available at: https://github.com/barbagroup/AeroPython/blob/master/styles/custom.css*/
@font-face {
font-family: "Computer Modern";
src: url('http://mirrors.ctan.org/fonts/cm-unicode/fonts/otf/cmunss.otf');
}
div.cell{
width:800px;
margin-left:16% !important;
margin-right:auto;
}
h1 {
font-family: 'Alegreya Sans', sans-serif;
}
h2 {
font-family: 'Fenix', serif;
}
h3{
font-family: 'Fenix', serif;
margin-top:12px;
margin-bottom: 3px;
}
h4{
font-family: 'Fenix', serif;
}
h5 {
font-family: 'Alegreya Sans', sans-serif;
}	
div.text_cell_render{
font-family: 'Alegreya Sans',Computer Modern, "Helvetica Neue", Arial, Helvetica, Geneva, sans-serif;
line-height: 135%;
font-size: 120%;
width:600px;
margin-left:auto;
margin-right:auto;
}
.CodeMirror{
font-family: "Source Code Pro";
font-size: 90%;
}
/* .prompt{
display: None;
}*/
.text_cell_render h1 {
font-weight: 200;
font-size: 50pt;
line-height: 100%;
color:#CD2305;
margin-bottom: 0.5em;
margin-top: 0.5em;
display: block;
}	
.text_cell_render h5 {
font-weight: 300;
font-size: 16pt;
color: #CD2305;
font-style: italic;
margin-bottom: .5em;
margin-top: 0.5em;
display: block;
}
.warning{
color: rgb( 240, 20, 20 )
}
</style>
<script>
MathJax.Hub.Config({
TeX: {
extensions: ["AMSmath.js"]
},
tex2jax: {
inlineMath: [ ['$','$'], ["\\(","\\)"] ],
displayMath: [ ['$$','$$'], ["\\[","\\]"] ]
},
displayAlign: 'center', // Change this to 'center' to center equations.
"HTML-CSS": {
styles: {'.MathJax_Display': {"margin": 4}}
}
});
</script>